# BRILLTagger
http://www.nltk.org/api/nltk.tag.html

In [3]:
import nltk

from nltk.tag import untag, RegexpTagger, BrillTaggerTrainer
from nltk.corpus import  conll2002

In [4]:
tags = [ tag for (word, tag) in conll2002.tagged_words('esp.train') ]
nltk.FreqDist(tags).max()

u'NC'

In [5]:
train_senteces = conll2002.tagged_sents('esp.train')
print len(train_senteces)
print train_senteces[0]
all_senteces = conll2002.tagged_sents()
print len(all_senteces)
print all_senteces[0]

8323
[(u'Melbourne', u'NP'), (u'(', u'Fpa'), (u'Australia', u'NP'), (u')', u'Fpt'), (u',', u'Fc'), (u'25', u'Z'), (u'may', u'NC'), (u'(', u'Fpa'), (u'EFE', u'NC'), (u')', u'Fpt'), (u'.', u'Fp')]
35651
[(u'Sao', u'NC'), (u'Paulo', u'VMI'), (u'(', u'Fpa'), (u'Brasil', u'NC'), (u')', u'Fpt'), (u',', u'Fc'), (u'23', u'Z'), (u'may', u'NC'), (u'(', u'Fpa'), (u'EFECOM', u'NP'), (u')', u'Fpt'), (u'.', u'Fp')]


In [17]:
training_data = train_senteces[:-100]
# baseline_data = senteces[-400:-100] #equivalent to CV data
gold_data = train_senteces[-100:]  #equivalent to test data
testing_data = [untag(s) for s in gold_data]

In [18]:
from nltk.tbl.template import Template
from nltk.tag.brill import Pos, Word
from nltk.tag import DefaultTagger

default_tagger = DefaultTagger('NC')

from nltk.tag import UnigramTagger

print "- Unigram Tagger"
unigram_tagger = UnigramTagger(training_data,backoff=default_tagger)
print "- Done."

- Unigram Tagger
- Done.


In [38]:
print "- Templates"
Template._cleartemplates() 
templates = [
    Template(Pos([-1])), 
    Template(Pos([-1]), Word([0])), 
    Template(Pos([-2])), 
    Template(Pos([-2]), Word([0])),
    Template(Pos([1])), 
]
print "- Creating Trainer"
tt = BrillTaggerTrainer(unigram_tagger, templates, trace=1)
print "- Begin Training"
tagger1 = tt.train(training_data, max_rules=1000)
print "- Done."

- Templates
- Creating Trainer
- Begin Training
TBL train (fast) (seqs: 8223; tokens: 261562; tpls: 5; min score: 2; min acc: None)
Finding initial useful rules...
    Found 10051 useful rules.
Selecting rules...
- Done.


In [39]:
tagger1.evaluate(gold_data)

0.9381541389153187

## Simple stupid test... that resulted in something 
Try to use the iob_sents to find the Named Entity:

1. Extract features
   - First letter uppercase?
   - Begin of chunk
   - pos[-1] tag
   - pos[1] tag
   - pos in chunk
   - pos in sentence



In [25]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

/Users/alextabora/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [40]:
def get_df(sentences):
    s_id = []
    s_word = []
    s_tag = []
    s_iob =[]
    s_pos = []

    for sent_num in range(len(sentences)):
        sent = sentences[sent_num]
        for pos in range(len(sent)):
            word = sent[pos]
            s_id.append(sent_num)
            s_word.append(word[0])
            s_tag.append(word[1])
            s_iob.append(word[2])
            s_pos.append(pos)
        
    df = pd.DataFrame({
            "sentence": s_id,
            "word": s_word,
            "tag": s_tag,
            "iob": s_iob,
            "pos": s_pos
        })
    
    return df
sent_df = get_df(conll2002.iob_sents('esp.train'))

### Features

In [41]:
from sklearn.preprocessing import LabelEncoder

def get_features(base_df):
    df = pd.DataFrame()
    # iob
    le_iob = LabelEncoder()
    df.loc[:,"iob"] = le_iob.fit_transform(base_df.iob)
    print list(le_iob.classes_)

    #tag
    le_tag = LabelEncoder()
    df.loc[:,"tag"] = le_tag.fit_transform(base_df.tag)
#     print list(le_tag.classes_)
    
    # Uppercase
    df.loc[:,"upper"] = base_df.word.apply(lambda x: x[0].isupper())
    
    # Pos
    df.loc[:,"pos"] = base_df.pos
    
    #first 
    df.loc[:,"first"] = base_df.pos.apply(lambda x: int(x == 0) )
    #size
    df.loc[:,"size"] = base_df.word.apply(lambda x: len(x))
    
    
    return df, le_iob, le_tag



feature_df, le_iob, le_tag = get_features(sent_df)
feature_df.head()

[u'B-LOC', u'B-MISC', u'B-ORG', u'B-PER', u'I-LOC', u'I-MISC', u'I-ORG', u'I-PER', u'O']


,iob,tag,upper,pos,first,size
0,0,28,True,0,1,9
1,8,20,False,1,0,1
2,0,28,True,2,0,9
3,8,21,False,3,0,1
4,8,12,False,4,0,1


### X and Y

In [61]:
#balancer
def balance_classes(df):

    classes = []
    for i in range(9):
        #binarizing
        if i == 8:
            class_df = df[df["iob"]==i]#[:75170]
            class_df.iob = 0
            classes.append(class_df )
        else:
            class_df = df[df["iob"]==i]#[:14000]
            class_df.iob = 1
            classes.append(class_df)
            

    newdf = pd.concat(classes)
        
    return newdf

neo_feature_df = balance_classes(feature_df)
neo_feature_df.iob.value_counts()

/Users/alextabora/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


0    231920
1     32795
Name: iob, dtype: int64

In [62]:
def getXY(df):
    X = df.drop("iob",1).values
    y = df.iob.values
    
    
    return X, y

X,y = getXY(neo_feature_df)

In [63]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state= 233, n_jobs=2)

#train
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10,shuffle=True, random_state= 233)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #train
    clf.fit(X_train, y_train)
    print "---------"
    print clf.score(X_train,y_train)
    print clf.score(X_test,y_test)
    

    

---------
0.978740193836
0.975899063161
---------
0.97847995534
0.978165608945
---------
0.978677232909
0.976692354186
---------
0.978698219885
0.975974614687
---------
0.978576495427
0.977296766395
---------
0.978794848978
0.975709266745
---------
0.978673125031
0.976351478977
---------
0.978563993217
0.977673680632
---------
0.978635348634
0.977484794681
---------
0.978538808952
0.978013675343


In [64]:
from sklearn.metrics import accuracy_score
preds = clf.predict(X_test)
accuracy_score(y_test, preds)

0.97801367534282801

In [65]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, preds)

array([[23037,   155],
       [  427,  2852]])

In [66]:
from sklearn.metrics import classification_report
target_names = ['NE', 'Other']
print(classification_report(y_test, preds, target_names=target_names))

             precision    recall  f1-score   support

         NE       0.98      0.99      0.99     23192
      Other       0.95      0.87      0.91      3279

avg / total       0.98      0.98      0.98     26471



## Tagged Sentence

In [67]:
#train all
print "- train all"
clf_final = RandomForestClassifier(n_estimators=100, random_state= 233, n_jobs=2)
clf_final.fit(X, y)
print "- Done."

- train all
- Done.


In [70]:
from nltk.tokenize import word_tokenize

raw = "El conductor del carro fue identificado como Miguel Perez que conducía en Brazil"

raw = "De conformidad con las leyes vigentes se realizaron los estudios en San Pedro Sula"
raw = "Debido a problemas en el sistema interno que lleva el Hospital Escuela, las citas están siendo programadas para el 2010"


tagged_test = tagger1.tag(word_tokenize(raw))
tagged_test

[('Debido', u'AQ'),
 ('a', u'SP'),
 ('problemas', 'NC'),
 ('en', u'SP'),
 ('el', u'DA'),
 ('sistema', 'NC'),
 ('interno', u'AQ'),
 ('que', u'PR'),
 ('lleva', u'VMI'),
 ('el', u'DA'),
 ('Hospital', 'NC'),
 ('Escuela', 'NC'),
 (',', u'Fc'),
 ('las', u'DA'),
 ('citas', 'NC'),
 ('est\xc3\xa1n', 'NC'),
 ('siendo', u'VSG'),
 ('programadas', u'AQ'),
 ('para', u'SP'),
 ('el', u'DA'),
 ('2010', u'Z')]

In [54]:
#convert tagged sentence into X format
def get_X_real(tagged_sent):
    X_real = []
    for word_num in range(len(tagged_sent)):
        #tag
        tag = le_tag.transform([tagged_sent[word_num][1]])[0]
        #upper
        upper = tagged_sent[word_num][0][0].isupper()
        #pos
        pos = word_num
        #first
        first = int(pos==0)
        #size
        size = len(tagged_sent[word_num][0])

        X_real.append([tag,upper,pos,first,size])
    return X_real
    
X_real = get_X_real(tagged_test)
X_real

[[1, True, 0, 1, 6],
 [39, False, 1, 0, 1],
 [27, False, 2, 0, 9],
 [39, False, 3, 0, 2],
 [4, False, 4, 0, 2],
 [27, False, 5, 0, 7],
 [1, False, 6, 0, 7],
 [34, False, 7, 0, 3],
 [46, False, 8, 0, 5],
 [4, False, 9, 0, 2],
 [27, True, 10, 0, 8],
 [27, True, 11, 0, 8],
 [4, False, 12, 0, 3],
 [27, False, 13, 0, 5],
 [27, False, 14, 0, 6],
 [51, False, 15, 0, 6],
 [1, False, 16, 0, 11],
 [39, False, 17, 0, 4],
 [4, False, 18, 0, 2],
 [58, False, 19, 0, 4]]

In [69]:
preds_real = clf_final.predict(X_real)
for i in range(len(preds_real)):
    if preds_real[i] == 1:
        print tagged_test[i][0]

Debido
Hospital
Escuela,


# TEST with data

In [52]:
import urllib2
import bs4 #beautiful soup
import pandas as pd
import numpy as np

from datetime import datetime
import pandas as pd
import numpy as np

articles = pd.read_csv("./files/articles.csv")
articles.shape

content = articles.loc[1].content   #1
soup = bs4.BeautifulSoup(content, 'html.parser')
# print soup.get_text()

raw = soup.get_text()
sentences = raw.split(".")
#Tag and prepare
for index, sent in enumerate(sentences):
    
    print 
    print "-- Sentence %i " %index
    print sent
    print
    
    if sent =="\n":
        continue
    
    
    
    

    tagged_sent = tagger1.tag( sent.split() )
#     print tagged_sent
    X_real = get_X_real(tagged_sent)


    #predict 
    preds_real = clf.predict(X_real)
#     print preds_real
    for i in range(len(preds_real)):
        if preds_real[i] == 1:
            print tagged_sent[i][0]


-- Sentence 0 
TEGUCIGALPA

TEGUCIGALPA

-- Sentence 1 
- Una misteriosa obstrucción del sistema de archivos y citas del Hospital Escuela, ha generado un caos entre los pacientes, debido a que todas las citas están siendo programadas hasta para cuatro meses del próximo año 2010 según las denuncias de los mismos empleados

Hospital
Escuela,

-- Sentence 2 

Debido a problemas en el sistema interno que lleva el Hospital Escuela, las citas están siendo programadas para el 2010

Debido
Hospital
Escuela,

-- Sentence 3 

Carmen Linares empleada del este centro hospitalario y afectada denunció el problema en el que se le están suspendiendo las citas a los pacientes, incluso hay rótulos de notificaciones anunciando que hasta segundo aviso se restablecerán están servicios

Carmen
Linares

-- Sentence 4 

“Nosotros estamos teniendo problemas con las citas, porque de forma extraña se dañaron las computadoras en donde se lleva el registro de las citas y los pacientes son los que están sufriendo 

# Sub-Conclusion...

### RFC can be used for Name Entitity identification task, given a POS tagged ... Although in practice it works fairly wel there are some cases it doesn't correctly identify.

## Next Steps
- Improve the POS tagger
- Improve the NE identifier
- Train a NER Classifier (one that classifies the identified NE )



In [ ]:
#reviewing how the sentences are tagged
# sent_num = 6

# print " ".join(conll2002.sents()[sent_num])
# print
# print conll2002.tagged_sents()[sent_num]

#find a sentence that starts with a "Art"
for index, sent in enumerate(conll2002.tagged_sents('esp.train')):
    if sent[0][1] == u"Art":
        print index
        print sent
        
        
        break




In [ ]:
print len(conll2002.iob_sents())
print len(conll2002.iob_sents('esp.testa'))
conll2002.iob_sents()[11758]

In [ ]:
feature_df.iob.hist()
plt.show()

In [ ]:
feature_df.iob.value_counts()

In [71]:
articles.head()

,id,date,title,content,tags
0,post-80480,"28 Dic, 2009 - 7:27 pm",Auditoría revela irregularidades en el Parlacen,"<div class=""article-post-content""><p><strong><...",None
1,post-80485,"28 Dic, 2009 - 7:32 pm",Suspendidas las citas en Hospital Escuela,"<div class=""article-post-content""><p><strong><...",None
2,post-80555,"28 Dic, 2009 - 8:25 pm",Mariscos contaminados alarman a los “porteños”,"<div class=""article-post-content""><p><strong><...",None
3,post-80559,"28 Dic, 2009 - 8:26 pm",Citan a 11 personas por vender pólvora,"<div class=""article-post-content""><p><strong><...",None
4,post-80578,"29 Dic, 2009 - 1:00 am",Con compra de granos se paliaría hambruna en e...,"<div class=""article-post-content""><p><strong><...",None


In [72]:
tagged_articles = pd.read_csv("./files/tagged_articles.csv")
tagged_articles.head()

,article_id,sent_id,tagged_sent
0,0,0,"[(u'GUATEMALA', 'NC')]"
1,0,1,"[(u'-', u'Fg'), (u'Una', u'DI'), (u'fiscalizac..."
2,0,2,"[(u'Auditor\xeda', 'NC'), (u'realizada', u'AQ'..."
3,0,3,"[(u'Los', u'DA'), (u'auditores', 'NC'), (u'de'..."
4,0,4,"[(u'Los', u'DA'), (u'informes', u'NC'), (u'ref..."


In [73]:
tagged_articles.loc[50:70]

,article_id,sent_id,tagged_sent
50,2,10,[]
51,3,0,"[(u'SAN', u'AQ'), (u'PEDRO', 'NC'), (u'SULA', ..."
52,3,1,"[(u'-', u'Fg'), (u'Hasta', u'SP'), (u'el', u'D..."
53,3,2,"[(u'Los', u'DA'), (u'operativos', u'AQ'), (u'c..."
54,3,3,"[(u'Resalt\xf3', u'VMI'), (u'que', u'CS'), (u'..."
55,3,4,"[(u'Indic\xf3', u'VMI'), (u'que', u'CS'), (u't..."
56,3,5,"[(u'Adelant\xf3', 'NC'), (u'que', u'CS'), (u'p..."
57,3,6,"[(u'La', u'DA'), (u'Polic\xeda', 'NC'), (u'Mun..."
58,3,7,"[(u'Record\xf3', u'VMI'), (u'que', u'CS'), (u'..."
59,3,8,"[(u'Revel\xf3', u'VMI'), (u'que', u'CS'), (u'l..."
